In [2]:
import os, glob
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from zipfile import ZipFile
import pdb

In [3]:
# get data location
#base_path = "/Users/nikkibytes/Documents/niblunc/BCP/BCP_DATA"#"/Users/nikkibytes/Documents/BCP_DATA"
basepath = "/Users/gracer/Google Drive/BCP/data"
#text4_files=glob.glob(os.path.join(base_path, "set*/*/*/*04.txt"))
#text9_files=glob.glob(os.path.join(base_path, "set*/*/*/*09.txt"))

# Read in the data from the zip files

In [4]:
list_of_files = {}
for (dirpath, dirnames, filenames) in os.walk(basepath):
    for filename in filenames:
        if filename.endswith('.zip'):
            #print(filename)
            tmppath=os.sep.join([dirpath, filename])
            #print(tmppath)
            with ZipFile(tmppath, 'r') as zipObj:
               # Get a list of all archived file names from the zip
               listOfFileNames = zipObj.namelist()
               # Iterate over the file names
               for fileName in listOfFileNames:
                   # Check filename endswith csv
                    if fileName.endswith('04.txt'):
                                           # Extract a single file from zip
                        zipObj.extract(fileName, os.path.join(basepath,'temp_txt'))
                    if fileName.endswith('09.txt'):
                                           # Extract a single file from zip
                        zipObj.extract(fileName, os.path.join(basepath,'temp_txt'))
infile = os.path.join(basepath,'temp_txt')

# Dictionary of the files we want

In [5]:
file_dict = {"set_04": {"mom":{}, "child": {}}, "set_09": {"mom": {}, "child": {}}}
file_dict['set_04']["mom"]["files"] = [x for x in glob.glob(os.path.join(infile,'*04.txt')) if "Mom" in x]
file_dict['set_09']["mom"]["files"] = [x for x in glob.glob(os.path.join(infile,'*09.txt')) if "Mom" in x]
file_dict['set_04']["child"]["files"] = [x for x in glob.glob(os.path.join(infile,'*04.txt')) if "Chil" in x]
file_dict['set_09']["child"]["files"] = [x for x in glob.glob(os.path.join(infile,'*09.txt')) if "Chil" in x]


# Concat

In [6]:
temp_list = []            
for file in file_dict["set_04"]["mom"]["files"]:
    temp_df =  pd.read_csv(file, sep="\t", encoding='latin1')
    temp_df=temp_df.drop([0])
    for val in temp_df["Participant ID"]:
        _id = str(val).lstrip("0").split("_")[0]
        temp_df.replace(val, _id, inplace=True)
    #print(updated_df["Participant ID"])
    temp_list.append(temp_df)
    #print(temp_df.shape)
dfm4_original = pd.concat(temp_list, ignore_index=True)
print("Final dataframe size: ", dfm4_original.shape)

dfm4_original.head()
dfm4_orignal = dfm4_original.sort_values(by="Participant ID")
dfm4_original.head()

Final dataframe size:  (360, 226)


,Project Abbreviation,Participant ID,Date of Intake,Project Name,Record Type,Participant Name,Gender,Date of Birth,Date of Entry,Day of Intake,...,Data Field 1 Descriptor,Data Field 1 Response,Data Field 2 Descriptor,Data Field 2 Response,Data Field 3 Descriptor,Data Field 3 Response,Data Field 4 Descriptor,Data Field 4 Response,Data Field 5 Descriptor,Data Field 5 Response
0,BCPMomMSTR,1995,06/06/2017,"Baby ConnectoMe Project, Mom MASTER, Q4 2017",1,NaN,3,NaN,12/14/2017,2,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
1,BCPMomMSTR,11370,05/30/2017,"Baby ConnectoMe Project, Mom MASTER, Q4 2017",1,NaN,3,NaN,11/21/2017,2,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
2,BCPMomMSTR,11370,08/23/2017,"Baby ConnectoMe Project, Mom MASTER, Q4 2017",1,NaN,3,NaN,11/21/2017,3,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
3,BCPMomMSTR,17541,05/30/2017,"Baby ConnectoMe Project, Mom MASTER, Q4 2017",1,NaN,3,NaN,01/02/2018,2,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
4,BCPMomMSTR,20784,12/13/2016,"Baby ConnectoMe Project, Mom MASTER, Q4 2017",1,NaN,3,NaN,12/14/2017,2,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN


In [7]:
temp_list = []            
for file in file_dict["set_09"]["mom"]["files"]:
    temp_df = pd.read_csv(file,encoding='latin1', sep="\t")
#     print("Loading file {}...........Dataframe size: {}".format(file.split("/")[-1], temp_df.shape))
    temp_df=temp_df.drop([0])
    for val in temp_df["Participant ID"]:
        _id = str(val).strip("0").strip(".").split("_")[0]
        #print(_id)
        temp_df.replace(val, _id, inplace=True)
    #print(updated_df["Participant ID"])
    temp_list.append(temp_df)
dfm9_original = pd.concat(temp_list, ignore_index=True)
dfm9_orignal = dfm9_original.sort_values(by="Participant ID")
print("Final dataframe size: ", dfm9_original.shape)
dfm9_original.head()


Final dataframe size:  (368, 171)


,Project Abbreviation,Participant ID,Date of Intake,FRU0100,FRU0200,FRU0300,FRU0400,FRU0500,FRU0600,FRU0700,...,MSC0300,MSC0400,MSC0500,MSC0600,MSC0700,MSC0800,MSC0900,MSC1000,GRW1300,GRS1300
0,BCPMomAP,11370,11/29/2017,1.500,0,0,0.500,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BCPMomAP,94794,12/03/2017,0.012,0,0,0,0.354,0,0,...,0,0,0.360,0,0,0,0,0,0,0
2,BCPMomAP,116769,12/19/2017,0,0,0,0,0,0,0,...,0.907,0,0,0,0,0,0,0,0,0
3,BCPMomAP,155866,02/19/2018,0,0,0,0.800,0.907,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BCPMomAP,156965,11/26/2017,0,0,0,0,0,0,0,...,0.250,0.630,0.117,0,0,0,0,0,0,0


# Saving

In [8]:
concat_filepath = os.path.join(basepath,'Concat','Mom_BCP_dataset4.txt')
dfm4_original.to_csv(concat_filepath, index=False, sep="\t", header=True)

In [9]:
concat_filepath = os.path.join(basepath,'Concat','Mom_BCP_dataset9.txt')
dfm9_original.to_csv(concat_filepath, index=False, sep="\t", header=True)

# Remove stupid fucking commas

In [10]:
for col in dfm4_original:
    if dfm4_original[col].dtype == np.object_:
        dfm4_original[col] = (dfm4_original[col].replace(',','.', regex=True))
        

# Join 04 and 09

## Find common elements

In [11]:
b=list(dfm9_original['Participant ID'])

In [12]:
a=list(dfm4_original['Participant ID'])

In [13]:
missmatch = list(set(a)-set(b))

In [15]:
common=list(set(a) & set(b))

In [16]:
common4=dfm4_original[dfm4_original['Participant ID'].isin(common)]

In [17]:
common9=dfm9_original[dfm9_original['Participant ID'].isin(common)]

In [18]:
total_df = common4.merge(common9.drop_duplicates(subset=['Project Abbreviation','Date of Intake']), how='left')

In [19]:
total_df=total_df.dropna(axis=1, how='all')

In [20]:
important=['Participant ID','VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700',
           'VEG0600','VEG0900','VEG0500','VEG0100','VEG0700','FRU0100','FRU0200','FRU0300','FRU0400',
           'FRU0500','FRU0600','FRU0700','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700',
           'Whole Grains (ounce equivalents)','DMF0100','DMR0100','DML0100','DMN0100','DMF0200',
           'DMR0200','DML0200','DML0300','DML0400','DCF0100','DCR0100','DCL0100','DCN0100','DYF0100',
           'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100','DOT0300','DOT0400','DOT0500',
           'DOT0600','DOT0100','MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
           'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200','MFF0100','MFL0100',
           'MFF0200','MSL0100','MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400',
           'MOF0500','MOF0600','MOF0700','VEG0700','MFF0100','MFL0100','MFF0200','MSL0100','MSF0100',
           'MOF0500','MOF0600','MOF0700','VEG0700','Sodium (mg)','Refined Grains (ounce equivalents)',
           'Added Sugars (by Total Sugars) (g)','% Calories from SFA','Energy (kcal)',
           'Total Polyunsaturated Fatty Acids (PUFA) (g)','Total Monounsaturated Fatty Acids (MUFA) (g)',
           'Total Saturated Fatty Acids (SFA) (g)']

In [21]:
complete=total_df[total_df.columns.intersection(important)].dropna()

In [22]:
cind=list(complete.index)

In [23]:
complete_df=total_df[total_df.index.isin(cind)]

In [24]:
missing_df=total_df[~total_df.index.isin(cind)]

In [25]:
concat_filepath = os.path.join(basepath,'Concat','Mom_BCP_datasetTOTAL.txt')
complete_df.to_csv(concat_filepath, index=False, sep="\t", header=True)

# Parameter dictionary

In [26]:
para_dict = {'hei_totveg': {'parameters':[1.1], 'name': 'HEIX1_TOTALVEG'}, 
             'hei_greensbeans': {'parameters':[0.2], 'name': 'HEIX2_GREEN_AND_BEAN'},
             'hei_totfruit': {'parameters':[0.8], 'name': 'HEIX3_TOTALFRUIT'},
             'hei_wholefruit': {'parameters':[0.4], 'name': 'HEIX4_WHOLEFRUIT'},
             'hei_wholegrains': {'parameters':[1.5], 'name': 'HEIX5_WHOLEGRAIN'},             
             'hei_dairy': {'parameters':[1.3], 'name': 'HEIX6_TOTALDAIRY'},
             'hei_totproteins': {'parameters':[2.5], 'name': 'HEIX7_TOTPROT'},
             'hei_seafoodplantprot': {'parameters':[0.8], 'name': 'HEIX8_SEAPLANT_PROT'},
             'hei_refinedgrains': {'parameters':[1.8,4.3], 'name': 'HEIX11_REFINEDGRAIN'},
             'hei_addedsugars': {'parameters':[6.5,26], 'name': 'HEIX12_ADDEDSUGARS'},
             'hei_SFA': {'parameters':[8,16], 'name': 'HEIX13_SATFATS'},
             'Fats': {'parameters':[1.2,2.5], 'name': 'HEIX9_FATTYACID'},
             'hei_sodium':{'parameters':[1.1,2.0],'name':'HEIX10_SODIUM'}
            }


In [27]:
hei_dict={'hei_totveg':
          ['VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700','VEG0600','VEG0900','VEG0500'],
          'hei_greensbeans':
          ['VEG0100','VEG0700'],
          'hei_totfruit':
          ['FRU0100','FRU0200','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
          'hei_wholefruit':
          ['FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
          'hei_wholegrains':
          ['Whole Grains (ounce equivalents)'],
          'hei_dairy':
          ['DMF0100','DMR0100','DML0100','DMN0100','DMF0200','DMR0200','DML0200',
                       'DML0300','DML0400','DCF0100','DCR0100','DCL0100','DCN0100','DYF0100',
                       'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100',
                       'DOT0300','DOT0400','DOT0500','DOT0600','DOT0100'],
          'hei_totproteins':
          ['MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
                             'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200',
                             'MFF0100','MFL0100','MFF0200','MSL0100',
                             'MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400','MOF0500',
                             'MOF0600','MOF0700','VEG0700'],
          'hei_seafoodplantprot':
          ['MFF0100','MFL0100','MFF0200','MSL0100','MSF0100','MOF0500','MOF0600','MOF0700','VEG0700'],
          'hei_sodium':
          ['Sodium (mg)'],
          'hei_refinedgrains':
          ['Refined Grains (ounce equivalents)'],
          'hei_addedsugars':
          ['Added Sugars (by Total Sugars) (g)'], 
          'ripctsfa': ['% Calories from SFA','Energy (kcal)'],
         'energy':
         ['Energy (kcal)'],
         'fats':
         ['Total Polyunsaturated Fatty Acids (PUFA) (g)','Total Monounsaturated Fatty Acids (MUFA) (g)',
         'Total Saturated Fatty Acids (SFA) (g)']}

In [28]:
hei_dict.keys()

dict_keys(['hei_totveg', 'hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_wholegrains', 'hei_dairy', 'hei_totproteins', 'hei_seafoodplantprot', 'hei_sodium', 'hei_refinedgrains', 'hei_addedsugars', 'ripctsfa', 'energy', 'fats'])

In [46]:
for key, value in hei_dict.items():
    print(key)
    if key in ['hei_totveg','hei_greensbeans','hei_totfruit', 'hei_wholefruit']:
        x=value
        complete_df[key] = complete_df[x].astype('float').sum(axis=1)
        complete_df[key] = complete_df[key]/2
    if key in ['hei_wholegrains','hei_refinedgrains']:
        x=value
        complete_df[key] = complete_df[x].astype('float')
    if key in ['hei_dairy']:
        x=value[:-1]
        tmp= complete_df[x].astype('float').sum(axis=1)
        y=value[-1]
        if y == 'DOT0100':
            tmp2=complete_df[y].astype('float')/3
            complete_df[key]=tmp+tmp2
        else:
            print('NO DAIRY MISSING DOT0100, needs to be last in list')
    if key in ['hei_totproteins']:
        x=value[:-1]
        tmp= complete_df[x].astype('float').sum(axis=1)
        y=value[-1]
        if y == 'VEG0700':
            tmp2=complete_df[y].astype('float')*2
            complete_df[key]=tmp+tmp2
        else:
            print('NO TOTAL PROTEIN MISSING VEG0700, needs to be last in list')
    if key in ['hei_seafoodplantprot']:
        x=value[:-1]
        tmp= complete_df[x].astype('float').sum(axis=1)
        y=value[-1]
        if y == 'VEG0700':
            tmp2=complete_df[y].astype('float')*2
            complete_df[key]=tmp+tmp2
        else:
            print('NO SEAFOOD AND PLANT PROTEIN MISSING VEG0700, needs to be last in list')
    if key in ['hei_sodium']:
        x=value
        complete_df[key] = complete_df[x].astype('float')/1000
    if key in ['hei_addedsugars']:
        x=value
        complete_df[key] = complete_df[x].astype('float')*4
    if key in ['ripctsfa']:
        x=value[0]
        y=value[1]
        complete_df[key] = complete_df[x].astype('float')*complete_df[y].astype('float')
    if key in ['energy']:
        x=value
        complete_df[key] = complete_df[x].astype('float')/1000

hei_totveg
hei_greensbeans
hei_totfruit
hei_wholefruit
hei_wholegrains
hei_dairy
hei_totproteins
hei_seafoodplantprot
hei_sodium
hei_refinedgrains
hei_addedsugars
ripctsfa
energy
fats


proc means data=hei0409togroup noprint;
  by cpartid;
  var rikcal hei_totveg hei_greensbeans hei_totfruit hei_wholefruit hei_wholegrains hei_dairy hei_totproteins hei_seafoodplantprot 
		ripfa rimfa risfa hei_sodium hei_refinedgrains hei_addedsugars ripctsfa;
  output out=dailyhei0409 sum=;
run;

# Grouping

In [47]:
interest = ['Participant ID','Energy (kcal)', 'hei_totveg', 'hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_wholegrains',
            'hei_dairy', 'hei_totproteins', 'hei_seafoodplantprot', 'Total Polyunsaturated Fatty Acids (PUFA) (g)',
            'Total Monounsaturated Fatty Acids (MUFA) (g)', 'Total Saturated Fatty Acids (SFA) (g)', 
            'hei_sodium', 'hei_refinedgrains', 'hei_addedsugars', 'ripctsfa']

In [56]:
dailyhei0409=complete_df[complete_df.columns.intersection(interest)].groupby(['Participant ID'])

In [59]:
dailyhei0409=dailyhei0409.describe()

In [60]:
dailyhei0409

Energy (kcal)                                                  \
                       count         mean          std       min         25%   
Participant ID                                                                 
107008                   2.0  1982.607000     0.000000  1982.607  1982.60700   
109504                   8.0  3059.293000   413.618509  2427.762  2904.13425   
116769                   3.0  2123.448333   267.614450  1822.925  2017.16550   
119470                   4.0  2558.334500   841.853045  1480.256  2225.12075   
131688                   2.0  1988.787000   450.873911  1669.971  1829.37900   
133077                   2.0  2967.716000     0.000000  2967.716  2967.71600   
133561                   9.0  2111.681000   441.034010  1522.375  1758.13100   
144877                   5.0  1990.006400   530.411438  1436.812  1587.85600   
155866                   2.0  1754.638000    53.257869  1716.979  1735.80850   
156965                   4.0  2193.550500   366.031205  1803.827  2002.03700   
167383                   5.0  2370.587600   826.497471  1363.961  1902.72000   
17541                    2.0  3118.359000     0.000000  3118.359  3118.35900   
176411                   4.0  2088.560000   364.390155  1702.913  1907.68625   
183117                   4.0  3026.450500   484.515576  2440.837  2735.14975   
1995                     2.0  2916.060000     0.000000  2916.060  2916.06000   
20784                    9.0  2168.482000   601.632030  1096.180  1721.77100   
223289                   4.0  2579.880250   462.807098  1940.045  2442.09650   
245640                   4.0  2402.539500   682.560691  1834.265  1949.84900   
249520                   3.0  3268.851000   282.784207  3005.833  3119.30650   
251117                   1.0  2630.541000          NaN  2630.541  2630.54100   
254818                   1.0  1991.709000          NaN  1991.709  1991.70900   
256619                   1.0   996.611000          NaN   996.611   996.61100   
257080                   3.0  2113.712667   633.186659  1663.945  1751.66750   
280884                   1.0  1659.319000          NaN  1659.319  1659.31900   
281070                   4.0  1845.642500   382.651533  1357.974  1633.16700   
290933                   1.0  1722.505000          NaN  1722.505  1722.50500   
300049                   2.0  3074.376500   308.872020  2855.971  2965.17375   
313655                   1.0  1940.271000          NaN  1940.271  1940.27100   
317427                   1.0  3064.002000          NaN  3064.002  3064.00200   
317472                   4.0  2962.027750   662.017743  2059.225  2671.99000   
...                      ...          ...          ...       ...         ...   
838504                   1.0  1375.774000          NaN  1375.774  1375.77400   
846789                   1.0  2002.031000          NaN  2002.031  2002.03100   
853992                   1.0  1836.464000          NaN  1836.464  1836.46400   
861379                   2.0  2274.159000   774.103734  1726.785  2000.47200   
867563                   3.0  1871.178333   200.899996  1639.199  1813.18350   
878596                   6.0  2686.888500   773.868928  1721.063  2263.77800   
884169                   2.0   983.058000     0.000000   983.058   983.05800   
886496                   1.0  1514.757000          NaN  1514.757  1514.75700   
887687                   1.0  1579.669000          NaN  1579.669  1579.66900   
892320                   1.0  3094.807000          NaN  3094.807  3094.80700   
895528                   1.0  2770.811000          NaN  2770.811  2770.81100   
897704                   2.0  2724.452500  1142.140793  1916.837  2320.64475   
900077                   3.0  2932.460333   587.002930  2575.736  2593.71300   
901945                   1.0  1964.563000          NaN  1964.563  1964.56300   
910908                   1.0  1590.027000          NaN  1590.027  1590.02700   
915257                   1.0  3369.950000          NaN  3369.950  3369.95000   
915363         

# Important functions

In [33]:
def adeq_check(inputt, output, parameter):
    if inputt in ['hei_totveg','hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_totproteins', 'hei_seafoodplantprot']:
        tmp = dfm9_original[inputt]/dfm4_original['energy']
        dfm9_original[output] = [5 if x >= parameter else 5*(x/parameter) for x in tmp]
    elif inputt in ['hei_wholegrains', 'hei_dairy']:
        if inputt == 'hei_wholegrains':
            tmp = dfm4_original[inputt]/dfm4_original['energy']
            dfm4_original[output] = [10 if x >= parameter else 10*(x/parameter) for x in tmp]
        else:
            tmp = dfm9_original[inputt]/dfm4_original['energy']
            dfm9_original[output] = [10 if x >= parameter else 10*(x/parameter) for x in tmp]
    elif inputt in ['Fats']:
        FARMIN=parameter[0]
        FARMAX=parameter[1]
        tmp=dfm4_original['Total Polyunsaturated Fatty Acids (PUFA) (g)']+dfm4_original['Total Monounsaturated Fatty Acids (MUFA) (g)']
        tmp2=tmp/dfm4_original['Total Saturated Fatty Acids (SFA) (g)']
        dfm4_original[output] = [10 if x > FARMAX else 0 if x <= FARMIN else 10*((x-FARMIN)/(FARMAX-FARMIN)) for x in tmp2]  


In [34]:
def mod_check(inputt, output, parameter):    
    if inputt in ['hei_sodium','hei_refinedgrains']:
        tmp = dfm4_original[inputt]/dfm4_original['energy']
        dfm4_original[output] = [0 if x >= parameter[1] else 10 if x <= parameter[0] else 10-(10*((x-parameter[0])/(parameter[1]-parameter[0]))) for x in tmp]
    if inputt in ['hei_addedsugars']:
        tmp= 100*dfm4_original[inputt]/dfm4_original['energy']
        dfm4_original[output] = [0 if x >= parameter[1] else 10 if x < parameter[0] else 10-(10*((x-parameter[0])/(parameter[1]-parameter[0]))) for x in tmp]
    if inputt in ['hei_SFA']:
        tmp= dfm4_original['% Calories from SFA']
        dfm4_original[output] = [0 if x > parameter[1] else 10 if x < parameter[0] else 10-(10*((x-parameter[0])/(parameter[1]-parameter[0]))) for x in tmp]
        

In [35]:
def check(x):
    for key,values in x.items():
        print('Calculating score for %s'%key)
        if key in ['hei_totveg','hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_totproteins', 'hei_seafoodplantprot',
                    'hei_wholegrains', 'hei_dairy','Fats']:
            if key == 'Fats':
                adeq_check(key, values['name'], values['parameters'])
            else:
                adeq_check(key, values['name'], values['parameters'][0])
        if key in ['hei_sodium','hei_refinedgrains','hei_addedsugars','hei_SFA']:
            mod_check(key, values['name'], values['parameters'])
    

In [36]:
check(para_dict)

Calculating score for hei_totveg
Calculating score for hei_greensbeans
Calculating score for hei_totfruit
Calculating score for hei_wholefruit
Calculating score for hei_wholegrains
Calculating score for hei_dairy
Calculating score for hei_totproteins
Calculating score for hei_seafoodplantprot
Calculating score for hei_refinedgrains
Calculating score for hei_addedsugars
Calculating score for hei_SFA
Calculating score for Fats
> <ipython-input-33-c0cfd56dce12>(14)adeq_check()
-> FARMIN=parameter[0]
(Pdb) q


BdbQuit: 